# Contextual AI LIME Text ExplainerFactory with Keras

This tutorial is similar to [lime_text_explainer.ipynb](lime_text_explainer.ipynb), but instead of a Naive Bayes model we attempt to generate explanations with a neural network implemented with Keras.

The neural network is a simple multi-layer CNN with GloVe embeddings. This tutorial requires you to download the pre-trained word embeddings from this [link](http://nlp.stanford.edu/data/glove.6B.zip) (caution - this link initiates a 822MB download).

The modelling/text processing portions of this tutorial are heavily borrowed from [this Keras blog](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html).

Like with other Contextual AI tutorials, the main steps for generating explanations are:

1. Get an explainer via the `ExplainerFactory` class
2. Build the text explainer
3. Call `explain_instance`

### Step 1: Import libraries

In [1]:
# Some auxiliary imports for the tutorial
import os
import sys
import random
import math
import numpy as np
from pprint import pprint
from sklearn import datasets
from sklearn.model_selection import train_test_split

import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, Layer, Activation, \
Conv1D, MaxPooling1D, Convolution1D, Dropout, BatchNormalization, Conv1D, Concatenate, Flatten
from keras.optimizers import Adam
from keras.initializers import Constant
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.contrib.learn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Set seed for reproducibility
np.random.seed(123456)

# Set the path so that we can import the ExplainerFactory
sys.path.append('../../')

# Main Contextual AI imports
import xai
from xai.explainer import ExplainerFactory

###################################################
# Set the directory to the GloVe embeddings here! #
###################################################
GLOVE_DIR = ''
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
HIDDEN_UNITS = 128

Using TensorFlow backend.


### Step 2: Load dataset and train a model

In this tutorial, we rely on the 20newsgroups text dataset, which can be loaded via sklearn's dataset utility. Documentation on the dataset itself can be found [here](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). To keep things simple, we will extract data for 3 topics - baseball, Christianity, and medicine.

Our target model is a CNN which ingest pre-trained word embeddings.

In [2]:
# Train on a subset of categories

categories = [
    'rec.sport.baseball',
    'soc.religion.christian',
    'sci.med'
]

raw_train = datasets.fetch_20newsgroups(subset='train', categories=categories)
print(list(raw_train.keys()))
print(raw_train.target_names)
print(raw_train.target[:10])
raw_test = datasets.fetch_20newsgroups(subset='test', categories=categories)
    
# Turn text into lowercase
raw_train_text = [doc.lower() for doc in raw_train.data]
y_train = raw_train.target
raw_test_text = [doc.lower() for doc in raw_test.data]
y_test = raw_test.target

# Tokenizer
tokenizer = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tokenizer.fit_on_texts(raw_train_text)
vocab_size = len(tokenizer.word_index)
word_index = tokenizer.word_index

# Convert string to index
train_sequences = tokenizer.texts_to_sequences(raw_train_text)
test_texts = tokenizer.texts_to_sequences(raw_test_text)

# Padding
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
test_data = pad_sequences(test_texts, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Convert to numpy array
X_train = np.array(train_data, dtype='float32')
X_test = np.array(test_data, dtype='float32')

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

y_train_onehot = to_categorical(y_train, num_classes=3)
y_valid_onehot = to_categorical(y_valid, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

['DESCR', 'target', 'data', 'filenames', 'target_names']
['rec.sport.baseball', 'sci.med', 'soc.religion.christian']
[1 0 2 2 0 2 0 0 0 1]


### Prepare the embedding matrix

In [3]:
# Prepare the embedding matrix
# Code comes from https://keras.io/examples/pretrained_word_embeddings/
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Define the model

In [4]:
# Prepare the model

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling1D(4)(x)
x = Conv1D(128, 5)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling1D(4)(x)
x = Conv1D(128, 5)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = Flatten()(x)

preds = Dense(y_train_onehot.shape[1], activation='softmax')(x)

model = Model(sequence_input, preds)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8100      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
batch_normalization_1 (Batch (None, 996, 128)          512       
_________________________________________________________________
activation_1 (Activation)    (None, 996, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 996, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 249, 128)          0         
__________

### Train the model

In [5]:
model.fit([X_train], y_train_onehot, epochs=100, batch_size=50,
          validation_data=([X_valid], y_valid_onehot))

Train on 1432 samples, validate on 358 samples
Epoch 1/100
1432/1432 [==============================] - 5s 3ms/step - loss: 2.1773 - acc: 0.3897 - val_loss: 2.4187 - val_acc: 0.3743
Epoch 2/100
1432/1432 [==============================] - 0s 264us/step - loss: 1.1169 - acc: 0.5740 - val_loss: 1.2449 - val_acc: 0.4749
Epoch 3/100
1432/1432 [==============================] - 0s 219us/step - loss: 0.7730 - acc: 0.6899 - val_loss: 0.7144 - val_acc: 0.6732
Epoch 4/100
1432/1432 [==============================] - 0s 217us/step - loss: 0.4197 - acc: 0.8156 - val_loss: 3.1116 - val_acc: 0.3575
Epoch 5/100
1432/1432 [==============================] - 0s 220us/step - loss: 0.2677 - acc: 0.8897 - val_loss: 0.5152 - val_acc: 0.7821
Epoch 6/100
1432/1432 [==============================] - 0s 218us/step - loss: 0.2428 - acc: 0.9064 - val_loss: 0.9468 - val_acc: 0.6732
Epoch 7/100
1432/1432 [==============================] - 0s 216us/step - loss: 0.1793 - acc: 0.9385 - val_loss: 3.4665 - val_acc: 0.5

### Step 3: Instantiate the explainer

Here, we will use the LIME Text ExplainerFactory.

In [6]:
explainer = ExplainerFactory.get_explainer(domain=xai.DOMAIN.TEXT)

### Step 4: Build the explainer

This initializes the underlying explainer object. We provide the `explain_instance` method below with the raw text - LIME's text explainer algorithm will conduct its own preprocessing in order to generate interpretable representations of the data. Hence we must define a custom `predict_fn` which takes a raw piece of text, vectorizes it using the trained `tokenizer`, and passes the vector into the Keras model to generate class probabilities. LIME uses `predict_fn` to query our neural network order to learn its behavior around the provided data instance.

In [7]:
def predict_fn(instance):
    # Convert string to index
    sequence = tokenizer.texts_to_sequences(instance)

    # Padding
    data = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

    # Convert to numpy array
    arr = np.array(data, dtype='float32')
    
    return model.predict([arr])

explainer.build_explainer(predict_fn)

### Step 5: Generate some explanations

In [8]:
exp = explainer.explain_instance(
    labels=[0, 1, 2],
    instance=raw_train.data[100],
    num_features=10
)

print('Label', raw_train.target_names[raw_train.target[100]])
pprint(exp)

/experiments/venv_codebook_defenses/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Label rec.sport.baseball
{0: {'confidence': 0.9999999,
     'explanation': [{'feature': 'game', 'score': 0.2595229194688601},
                     {'feature': 'again', 'score': 0.18668745076997575},
                     {'feature': 'Yankees', 'score': 0.1860972771358493},
                     {'feature': 'pitches', 'score': 0.15736066007125038},
                     {'feature': 'Liberalizer', 'score': 0.1347915665789044},
                     {'feature': 'can', 'score': 0.12968895498952704},
                     {'feature': 'think', 'score': 0.11919896484535476},
                     {'feature': 'am', 'score': 0.11277455237479057},
                     {'feature': 'I', 'score': -0.05480655587778587},
                     {'feature': 'believe', 'score': -0.043605727115050126}]},
 1: {'confidence': 2.9914535e-09,
     'explanation': [{'feature': 'game', 'score': -0.001329492360436989},
                     {'feature': 'going', 'score': -0.0012736316324735816},
                     {'feat

### Step 6: Save and load the explainer

Like with the LIME tabular explainer, we can save and load the explainer via `load_explainer` and `save_explainer` respectively.

In [9]:
# Save the explainer somewhere

explainer.save_explainer('artefacts/lime_text_keras.pkl')

In [10]:
# Load the saved explainer in a new ExplainerFactory instance

new_explainer = ExplainerFactory.get_explainer(domain=xai.DOMAIN.TEXT, algorithm=xai.ALG.LIME)
new_explainer.load_explainer('artefacts/lime_text_keras.pkl')

exp = new_explainer.explain_instance(
    instance=raw_train.data[20],
    labels=[0, 1, 2],
    num_features=5
)

print('Label', raw_train.target_names[raw_train.target[20]])
pprint(exp)

/experiments/venv_codebook_defenses/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Label rec.sport.baseball
{0: {'confidence': 1.0,
     'explanation': [{'feature': 'baseball', 'score': 0.20855838996714443},
                     {'feature': 'stadium', 'score': 0.13433699819432926},
                     {'feature': 'football', 'score': 0.07024692251378775},
                     {'feature': 'in', 'score': -0.031260284138860533},
                     {'feature': 'with', 'score': -0.03063213505227814}]},
 1: {'confidence': 3.4470056e-13,
     'explanation': [{'feature': 'baseball', 'score': -0.009481833034551843},
                     {'feature': 'the', 'score': -0.007167239192790489},
                     {'feature': 'multipurpose', 'score': 0.004503424932271883},
                     {'feature': 'It', 'score': 0.004496400507397244},
                     {'feature': 'let', 'score': 0.004477692674225241}]},
 2: {'confidence': 3.5067134e-14,
     'explanation': [{'feature': 'baseball', 'score': -0.1942792635465446},
                     {'feature': 'stadium', 'score': -0.